In [1]:
#导入必须库
import json
import os

预处理：将所有数据集进行聚合，重新编号

In [2]:
#打开提取的题目json
with open('minif2f.jsonl', 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
#将二阶分类设置成minif2f
for i in data:
    i['category'] = 'minif2f'
with open('aime_amc_new.jsonl','r') as f:
    data2 = [json.loads(l) for l in f.readlines()]
#将二阶分类设置成aime_amc
for i in data2:
    i['category'] = 'aime_amc'
#合并两个json
data.extend(data2)

In [3]:
amc=0
aime=0
for item in data:

    if 'amc' in item['name'].lower():
        amc+=1
    if 'aime' in item['name'].lower():
        aime+=1
    
print(amc)
print(aime)

384
210


In [49]:
#以下是gpt构造器_单个subgoal生成
import openai
from openai import AzureOpenAI
import json
clientT = AzureOpenAI(
    api_key=
    azure_endpoint='https://hkust.azure-api.net',
    api_version="2023-05-15"
)

save_pth='new_dataset/'

class gpt_interface:
    def __init__(self):
        openai.api_base = "https://api2.aigcbest.top/v1"
        self.client = openai.Client(api_key=)
        try:
            with open(save_pth+'history_2.json','r') as f:
                self.history2=json.load(f)
        except:
            self.history2={}
        try:
            with open(save_pth+'history_3.json','r') as f:
                self.history3=json.load(f)

        except:
            
            self.history3 = {}
    def get_response(self,question,id):
        
        messages=[
            {
                "role": "system",
                "content": '''
You are a math assistant. Please give me all the subgoals of the proof based on the question below. You can analysis the probelm as you want. But you need to use [1][2][3]... to mark the subgoals.
theorem aime_problem (hx : x > 1) (hy : y > 1) 
  (h1 : (log (y ^ x)) / (log x) = 10)
  (h2 : (log (x ^ (4 * y))) / (log y) = 10) :
  x * y = 25 := by
Subgoals:
```subgoals
[1](x * log y) / (log x) = 10
[2]((x * log y) / (log x)) * ((4 * y * log x) / (log y)) = 100
[3](4 * x * y) * ((log y * log x) / (log x * log y)) = 100
[4]4 * x * y = 100
```
Now, please give me all the subgoals of this problem:
'''
            },
            {
                "role": "user",
                "content": question
            }
        ]
        if question in self.history2:
            messages.append(
             {   "role": "system",
                "content": self.history2[question]})
            return self.history2[question],messages,id
        #不在，则调用gpt

        response = clientT.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
        result_str=(response.choices[0].message.content)
        self.history2[question]=result_str
        messages.append(
            {   "role": "system",
                "content": result_str})
        self.save()
        return result_str,messages,id
    def get_details(self,message,indexs,id):
        if id in self.history3:
            return self.history3[id],id
        message.append({'role':'user','content':'Please help me refine the proof for the following subgoals: %s'%indexs})
        response = clientT.chat.completions.create(
        model="gpt-4o",
        messages=message
    )
        result_str=(response.choices[0].message.content)
        self.history3[id]=result_str
        self.save()
        return result_str,id


    def save(self):
      with open(save_pth+'history_2.json','w') as f:
          json.dump(self.history2,f)
      with open(save_pth+'history_3.json','w') as f:
          json.dump(self.history3,f)

In [40]:
#以下是多线程构造器
import concurrent.futures
def parallel_requests(func,prompts_list,max_process=16):
    results = [0]*len(prompts_list)
    # 使用线程池来并行处理请求
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_process) as executor:
        # 提交所有的请求到线程池
        futures = [executor.submit(func, item[0],item[1],id) for id,item in enumerate(prompts_list)]
        # 收集所有请求的结果
        for future in concurrent.futures.as_completed(futures):
            try:
                result,id = future.result()
                results[id]= result
            except Exception as exc:
                results.append({"error": str(exc)})

    return results

In [50]:
test1000=gpt_interface()

#产生一个gpt对象

In [7]:
#构造题目提问序列
prompts=[]
for item in data:
    prompts.append(item['informal_prefix'])

In [ ]:
prompt_list=parallel_requests(test1000.get_details,prompts2,64)


In [38]:
#循环扫描，获得所有的subgoal
sub_goals_list=[]
global_id=0
for item in prompt_list:
    #看看一共有几个subgoals
    id = 1
    for i in range(1,100):
        if '[%d]'%i in item[0]:
            id+=1
            global_id+=1
            sub_goals_list.append((item[1],'[%d]'%i))
        else:
            break
    
len(sub_goals_list)

4331

In [64]:
subgoals=parallel_requests(test1000.get_details,sub_goals_list,8)
test1000.save()
with open(save_pth+'subgoals.json','w') as f:
    json.dump(subgoals,f)

4331

In [65]:
global_id=0
spt=[]
for item in prompt_list:
    #看看一共有几个subgoals
    id = 1
    temp=[]
    for i in range(1,100):
        if '[%d]'%i in item[0]:
            temp.append(subgoals[global_id])
            id+=1
            global_id+=1
        else:
            break
    spt.append(temp)

In [68]:
for i in range(len(data)):
    data[i]['informal_proof_root']=prompt_list[i][0]
    data[i]['informal_proof_subgoals']=spt[i]

In [69]:
with open(save_pth+'super_dataset.json','w') as f:
    json.dump(data,f)

In [61]:
clientT = openai.Client()

In [44]:
with open(save_pth+'subgoals.json','w') as f:
    json.dump(subgoals,f)

In [48]:
#回写题目
for i in range(len(data)):
    data[i]['formal_statement']+='\n/-%s-/\n'%prompt_list[i][0]
with open('aime_amc_orihint.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item)+'\n')

In [181]:
# 生成幂集
import re
import itertools
def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))
def seg(text,indexs,detail_list):
    split_text = re.split(r'\[\d+\]', text)
    output=split_text[0]
    for i in range(len(detail_list)):
        output+='[%d]'%(i+1)
        if i in indexs:
            output+='\n'+refine(detail_list[i])+'\n'
        else:
            output+=split_text[i+1]
    return output
def refine(text): #只保留公式
    text=text.replace('\n',' ')
    outs=[]
    formulas = re.findall(r'\\[\s\S]*?\\]', text)
    for item in formulas:
        outs.extend(re.findall(r'\\\[[\s\S]*?\\]', item, re.DOTALL))
    if len(outs)==0:
        return text
    return '\n'.join(outs)

In [201]:
#super_select_data_maker
from tqdm import tqdm
super_set=[]
for item in tqdm(data):
    for ps in powerset(range(len(item['informal_proof_subgoals']))):
        result=seg(item['informal_proof_root'],ps,item['informal_proof_subgoals'])
        # print(result)
        new_obj=item.copy()
        new_obj['formal_statement']+='\n/-%s-/\n'%result
        new_obj['name']+=('_%s'%str(ps)).replace(' ','')
        del new_obj['informal_proof_subgoals']
        del new_obj['informal_proof_root'] #节约空间
        super_set.append(new_obj)
    

  0%|          | 0/961 [00:00<?, ?it/s]

100%|██████████| 961/961 [01:01<00:00, 15.52it/s]


In [203]:
for item in super_set:
    if item['split']=='valid':
        print(item)
        break

{'name': 'amc12a_2019_p21_()', 'split': 'valid', 'informal_prefix': '/-- Let $z=\\frac{1+i}{\\sqrt{2}}.$What is $\\left(z^{1^2}+z^{2^2}+z^{3^2}+\\dots+z^{{12}^2}\\right) \\cdot \\left(\\frac{1}{z^{1^2}}+\\frac{1}{z^{2^2}}+\\frac{1}{z^{3^2}}+\\dots+\\frac{1}{z^{{12}^2}}\\right)?$\n\n$\\textbf{(A) } 18 \\qquad \\textbf{(B) } 72-36\\sqrt2 \\qquad \\textbf{(C) } 36 \\qquad \\textbf{(D) } 72 \\qquad \\textbf{(E) } 72+36\\sqrt2$ Show that it is \\textbf{(C) }36.-/\n', 'formal_statement': 'theorem amc12a_2019_p21 (z : ℂ) (h₀ : z = (1 + Complex.I) / Real.sqrt 2) :\n  ((∑ k : ℤ in Finset.Icc 1 12, z ^ k ^ 2) * (∑ k : ℤ in Finset.Icc 1 12, 1 / z ^ k ^ 2)) = 36 := by\n\n/-To solve this problem, we need to analyze and compute the value of the given expression. Here are the subgoals to reach the solution:\n\n[1] **Determine the properties of \\( z \\):** \n   - Compute \\( |z| \\) and the argument of \\( z \\), which will help in understanding the pattern in powers of \\( z \\).\n   - Prove that \\

In [204]:
# 保存jsonl
with open(save_pth+'aime_amc_super.jsonl', 'w') as f:
    for item in super_set:
        f.write(json.dumps(item)+'\n')